In [1]:
import numpy as np
from numerical_methods.linear_iter_methods import jacobi, gauss_seidel

In [2]:
A = np.array([[10, -1, 2, 0], [-1, 11, -1, 3], [2, -1, 10, -1], [0, 3, -1, 8]])
A

array([[10, -1,  2,  0],
       [-1, 11, -1,  3],
       [ 2, -1, 10, -1],
       [ 0,  3, -1,  8]])

In [3]:
b = np.array([[6, 25, -11, 15]]).T
b

array([[  6],
       [ 25],
       [-11],
       [ 15]])

In [4]:
x0 = np.zeros(shape=b.shape)
x = np.zeros(shape=x0.shape)

In [5]:
jacobi(A, b, x0, max_iter=20)

(array([[ 0.99999994],
        [ 2.0000001 ],
        [-1.00000008],
        [ 1.00000011]]),
 20)

In [6]:
jacobi(A, b, x0, max_iter=20)

(array([[ 0.99999994],
        [ 2.0000001 ],
        [-1.00000008],
        [ 1.00000011]]),
 20)

In [7]:
gauss_seidel(A, b, x0, max_iter=20)

(array([[ 1.],
        [ 2.],
        [-1.],
        [ 1.]]),
 9)